In [130]:
# !pip install scikit-learn
# !pip install pandas
# !unzip data_numpy

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
import torch.nn.functional as F

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
chunk = np.load('data_numpy/0.npy')
labels = np.load('data_numpy/lb_0.npy', allow_pickle=True)
for i in [1, 2, 3, 4, 5]:
    b = np.load('data_numpy/'+str(i)+'.npy')
    chunk = np.concatenate((chunk, b), axis = 0)
    del b
    b = np.load('data_numpy/lb_'+str(i)+'.npy', allow_pickle=True)
    labels = np.concatenate((labels, b), axis = 0)
    del b

In [7]:
mydict = {'star': 0, 'qso': 1, 'galaxy': 2}
labels = np.array([mydict[key] for key in labels])

In [5]:
valid = np.load('data_numpy/6.npy')
valid_lbl = np.load('data_numpy/lb_6.npy', allow_pickle=True)
valid_lbl = np.array([mydict[key] for key in valid_lbl])

In [10]:
chunk = np.log(1.0*(chunk>=0)*chunk+1) - 1.0*np.log(-1.0*(1.0*(chunk<0)*chunk-1))
valid = np.log(1.0*(valid>=0)*valid+1) - 1.0*np.log(-1.0*(1.0*(valid<0)*valid-1))

In [9]:
mean = np.mean(chunk, axis=0)
std = np.std(chunk, axis=0)

In [12]:
np.save('all_mean.npy', mean)
np.save('all_std.npy', std)

In [3]:
mean = np.load('all_mean.npy')
std = np.load('all_std.npy')

In [12]:
chunk = (chunk-mean)/std
valid = (valid-mean)/std

In [4]:
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        
        # Define the layers
        self.fc1 = nn.Linear(2600, 1000)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(1000)
        
        self.fc2 = nn.Linear(1000, 500)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(500)
        
        self.fc3 = nn.Linear(500, 100)
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm1d(100)
        
        self.fc4 = nn.Linear(100, 50)
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm1d(50)
        
        self.fc5 = nn.Linear(50, 3)
        # self.logsoftmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.bn1(self.relu1(self.fc1(x)))
        x = self.bn2(self.relu1(self.fc2(x)))
        x = self.bn3(self.relu3(self.fc3(x)))
        x = self.bn4(self.relu4(self.fc4(x)))
        x = self.fc5(x)
        # x = self.logsoftmax(self.fc5(x))
        return x

In [5]:
model = MLPModel()
model.to(device);
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, betas=(0.9, 0.999), eps=1e-07)

In [101]:
model.train()
loss_list = []
val_list = []
min_val = 100000
weight = torch.tensor([0.1, 2, 0.8])
weight = weight.to(device, dtype=torch.float32)
for ep in range(500):
    tot_loss = 0
    model.train()
    for i in range(0, len(labels), 500):
        old_dict = model.state_dict()
        data = chunk[i:i+500]
        lbs = labels[i:i+500]
        
        data = torch.from_numpy(data.astype('float32'))
        data = data.to(device, dtype=torch.float32)
        lbs = torch.from_numpy(lbs)
        lbs = lbs.to(device)

        output = model(data)
        del data
        optimizer.zero_grad()
        loss = F.cross_entropy(output, lbs, weight)
        tot_loss = tot_loss + loss
        loss.backward()
        optimizer.step()
        del lbs
        del output

    valid_loss = 0
    model.eval()
    for i in range(0, len(valid), 500):
        data = valid[i:i+500]
        lbs = valid_lbl[i:i+500]

        data = torch.from_numpy(data.astype('float32'))
        data = data.to(device, dtype=torch.float32)
        lbs = torch.from_numpy(lbs)
        lbs = lbs.to(device)
        output = model(data)
        output = torch.argmax(output, axis=1)
        wgt = 1.0*lbs; wgt[wgt==0] = 0.1;wgt[wgt==1] = 2;wgt[wgt==2] = 1
        error = torch.sum(wgt*(output-lbs)*(output-lbs))/1000
        # loss = F.cross_entropy(output, lbs, weight)
        del data, lbs, output
        valid_loss = valid_loss + error
    val_list.append(valid_loss)
    
    if valid_loss < min_val:
        torch.save(model.state_dict(), "models/best_4")
        min_val = 1.0*valid_loss
    # print(valid_loss)


    if len(loss_list)>10:
        ave = sum(loss_list[-10:])/10
        if tot_loss > ave:
            loss_list.append(tot_loss)
            print(tot_loss)
            model.load_state_dict(old_dict)
            break
    

    loss_list.append(tot_loss)
    print(ep, "  ", tot_loss, valid_loss)

0    tensor(116.7611, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8881, device='cuda:0')
1    tensor(86.6275, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.9085, device='cuda:0')
2    tensor(74.2898, device='cuda:0', grad_fn=<AddBackward0>) tensor(1.2045, device='cuda:0')
3    tensor(66.2443, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.6329, device='cuda:0')
4    tensor(59.9163, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.5791, device='cuda:0')
5    tensor(55.5518, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.6680, device='cuda:0')
6    tensor(52.3829, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.7440, device='cuda:0')
7    tensor(50.0778, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.5911, device='cuda:0')
8    tensor(48.5570, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.6510, device='cuda:0')
9    tensor(45.5650, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.7255, device='cuda:0')
10    tensor(43.8377, device='cuda:0', grad_fn=<AddBackward

In [ ]:
test= np.load('test.npy')
test_lbl = np.load('test_lbl.npy', allow_pickle=True)
test_lbl = np.array([mydict[key] for key in test_lbl])
test = np.log(1.0*(test>=0)*test+1) - 1.0*np.log(-1.0*(1.0*(test<0)*test-1))
test = (test-mean)/std

In [27]:
# test_lbl.shape

In [28]:
# gaga = model.state_dict()
model.load_state_dict(torch.load('models/best_4'))
# model.load_state_dict(gaga)
model.eval()
glist = []
aa = []; bb = []; cc = [];
for i in range(0, len(test), 500):
    data = test[i:i+500]
    lbs = test_lbl[i:i+500]

    data = torch.from_numpy(data.astype('float32'))
    data = data.to(device, dtype=torch.float32)
    lbs = torch.from_numpy(lbs)
    lbs = lbs.to(device)
    # weight = torch.tensor([0.2, 1, 1])
    # weight = weight.to(device, dtype=torch.float32)

    output = model(data)
    glist.append(torch.sum(torch.argmax(output, axis=1) == lbs))
    # print(torch.sum(torch.argmax(output, axis=1) == lbs))
    aa.append(torch.sum(torch.argmax(output, axis=1)[lbs==1] == 1)*100/torch.sum(lbs==1))
    bb.append(torch.sum(torch.argmax(output, axis=1)[lbs==2] == 2)*100/torch.sum(lbs==2))
    cc.append(torch.sum(torch.argmax(output, axis=1)[lbs==0] == 0)*100/torch.sum(lbs==0))
    # break

In [33]:
sum(glist)/len(glist)*1/5
# sum(cc)/len(aa)
# len(glist)

tensor(97.2275, device='cuda:0')

In [108]:
print(torch.sum(torch.argmax(output, axis=1) == lbs))
torch.argmax(output, axis=1), lbs

tensor(483, device='cuda:0')


(tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2,
         0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
         2, 0, 0, 1, 0, 2, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0,
         0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
         2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0,
         0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0,
         0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0,
         0, 1, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 1, 0, 0,
         0, 0, 2, 2, 0, 0, 2

In [121]:
torch.sum(torch.argmax(output, axis=1)[lbs==1] == 1)*100/torch.sum(lbs==1)
torch.sum(torch.argmax(output, axis=1)[lbs==2] == 2)*100/torch.sum(lbs==2)
torch.sum(torch.argmax(output, axis=1)[lbs==0] == 0)*100/torch.sum(lbs==0)
# torch.argmax(output, axis=1)[lbs==2]
# torch.argmax(output, axis=1)[lbs==1]

tensor(98.3254, device='cuda:0')